In [5]:
import os

import pandas as pd
from neo4j import GraphDatabase

USERNAME = os.getenv("NEO4J_USERNAME", "neo4j")
PASSWORD = os.getenv("NEO4J_PASSWORD", "password")


class Neo4jConnection:
    # Function retrieved from:
    # https://towardsdatascience.com/create-a-graph-database-in-neo4j-using-python-4172d40f89c4

    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)

    def close(self):
        if self.__driver is not None:
            self.__driver.close()

    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response


conn = Neo4jConnection(
    uri="bolt://localhost:11005", 
    user=USERNAME,              
    pwd=PASSWORD
)

In [4]:
# Create constraints on the customer, country, and film nodes
conn.query('CREATE CONSTRAINT countries IF NOT EXISTS FOR (c:Country) REQUIRE c.name IS UNIQUE')
conn.query('CREATE CONSTRAINT customers IF NOT EXISTS FOR (c:Customer) REQUIRE c.id IS UNIQUE')
conn.query('CREATE CONSTRAINT films IF NOT EXISTS FOR (f:Film) REQUIRE f.id IS UNIQUE')

[]

In [30]:
# Note that to execute the below query, you will need to find the location of the import data or enable
# importing CSV data from any file location.
# To find the location of the import directory, run the following query:
#
# CALL dbms.listConfig()
# YIELD name, value
# WHERE name CONTAINS 'server.directories.import'
# RETURN name, value
# ORDER BY name
# LIMIT 20;
#

INSERT_COUNTRIES = '''
        WITH "file:///world_economic_data.csv" AS uri
        LOAD CSV WITH HEADERS FROM uri AS row
        MERGE (c:Country {name: row.name})
        SET c.gdp_per_capita = row.gdp_per_capita
        SET c.population_density = row.population_density
        SET c.population = row.population
        SET c.literacy_rate = row.literacy_rate
        SET c.access_to_electricity = row.access_to_electricity
        SET c.region = row.region
'''

conn.query(INSERT_COUNTRIES)

[]

## Extract Sakila Database Data

To extract data from the MySQL dataset, the following queries were executed against the database

```sql
SELECT film.film_id, release_year, rental_duration, rental_rate, length, replacement_cost, rating, \
    category.name AS category_name FROM film JOIN film_category ON film_category.film_id=film.film_id \
     JOIN category ON film_category.category_id=category.category_id \
    INTO OUTFILE '/var/lib/mysql-files/sakila_films.csv' \
    FIELDS TERMINATED BY ',' ENCLOSED BY '"' LINES TERMINATED BY '\n';

SELECT ID, country FROM customer_list INTO OUTFILE '/var/lib/mysql-files/sakila_customers.csv' \
    FIELDS TERMINATED BY ',' ENCLOSED BY '"' LINES TERMINATED BY '\n';

SELECT rental_id, customer_id, film_id, inventory.inventory_id, rental_date, return_date FROM rental \
    JOIN inventory ON rental.inventory_id=inventory.inventory_id INTO OUTFILE \
    '/var/lib/mysql-files/sakila_rentals.csv' FIELDS TERMINATED BY ',' \
    ENCLOSED BY '"' LINES TERMINATED BY '\n';
```

Once the above queries were executed, the files were moved to the data directory, where they can be found now :-).

Note that I did manually add the headers for the outputted CSV files...

In [35]:
INSERT_FILMS = '''
    WITH "file:///sakila_films.csv" AS uri
    LOAD CSV WITH HEADERS FROM uri AS row
    MERGE (f:Film {id: row.film_id})
    SET f.release_year = row.release_year
    SET f.rental_duration = row.rental_duration
    SET f.rental_rate = row.rental_rate
    SET f.length = row.length
    SET f.replacement_cost = row.replacement_cost
    SET f.rating = row.rating
    SET f.category_name = row.category_name

    WITH f, row
    MATCH (r:Rental {film_id: row.film_id})
    MERGE (f)-[:LENT]->(r)
'''

conn.query(INSERT_FILMS)

[]

In [31]:
INSERT_CUSTOMERS = '''
    WITH "file:///sakila_customers.csv" AS uri
    LOAD CSV WITH HEADERS FROM uri AS row
    MERGE (c:Customer {id: row.id})
    SET c.country = row.country

    // connect customer to country
    WITH c, row
    MATCH (country:Country {name: row.country})
    MERGE (c)-[:LIVES_IN]->(country)
'''

conn.query(INSERT_CUSTOMERS)

[]

In [27]:
INSERT_RENTALS = '''
    WITH "file:///sakila_rentals.csv" AS uri
    LOAD CSV WITH HEADERS FROM uri AS row
    MERGE (r:Rental {id: row.rental_id})
    SET r.customer_id = row.customer_id
    SET r.film_id = row.film_id
    SET r.rental_date = row.rental_date
    SET r.return_date = row.return_date
 
    // connect rental to customer
    WITH r, row
    MATCH (c:Customer {id: row.customer_id})
    MERGE (c)-[:LENT]->(r)

    // connect rental to film
    WITH r, row
    MATCH (f:Film {id: row.film_id})
    MERGE (f)-[:BORROWED]->(r)
'''

conn.query(INSERT_RENTALS)

[]

In [32]:
INSERT_RENTAL_EDGES = '''
    MATCH (c:Customer)
    MATCH (r:Rental)
    WHERE c.id = r.customer_id
    MERGE (c)-[:RENTED]->(r)
    RETURN COUNT(*) AS count
'''

conn.query(INSERT_RENTAL_EDGES)

[<Record count=16044>]

In [ ]:
df_customers = pd.read_csv('../data/customers.csv')

In [34]:
df_films = pd.read_csv("../data/sakila_films.csv")
df_films.head()

,film_id,release_year,rental_duration,rental_rate,length,replacement_cost,rating,category_name
0,19,2006,6,0.99,113,20.99,PG,Action
1,21,2006,3,4.99,129,17.99,R,Action
2,29,2006,5,2.99,168,11.99,NC-17,Action
3,38,2006,6,0.99,68,25.99,NC-17,Action
4,56,2006,6,2.99,129,15.99,G,Action
